## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-07-03-56-00 +0000,wsj,Senate Republicans Reject Measure to Block Mil...,https://www.wsj.com/world/americas/senate-repu...
1,2025-11-07-03-53-52 +0000,nypost,President Trump meets ‘hero’ Long Island cop c...,https://nypost.com/2025/11/06/us-news/presiden...
2,2025-11-07-03-47-05 +0000,nyt,San Francisco Sees Nancy Pelosi as a Tireless ...,https://www.nytimes.com/2025/11/06/us/nancy-pe...
3,2025-11-07-03-46-19 +0000,bbc,Elon Musk's $1tn pay deal approved by Tesla sh...,https://www.bbc.com/news/articles/cwyk6kvyxvzo...
4,2025-11-07-03-34-19 +0000,nyt,Testosterone and Women’s Sex Drive,https://www.nytimes.com/2025/11/06/world/testo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2334/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
9,trump,43
63,will,15
189,new,15
193,year,14
367,shutdown,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
24,2025-11-07-02-15-00 +0000,wsj,"Republican Rep. Elise Stefanik, an ally of Pre...",https://www.wsj.com/politics/elections/elise-s...,127
34,2025-11-07-02-00-00 +0000,wsj,Allies of President Trump and Health Secretary...,https://www.wsj.com/politics/elections/trump-r...,100
248,2025-11-06-17-38-00 +0000,wsj,Many GOP lawmakers are saying no—for now—to Pr...,https://www.wsj.com/politics/policy/senate-rep...,99
177,2025-11-06-20-57-06 +0000,latimes,Supreme Court rules Trump may remove transgend...,https://www.latimes.com/politics/story/2025-11...,95
178,2025-11-06-20-54-00 +0000,wsj,Employers said they cut more than a million jo...,https://www.wsj.com/economy/jobs/planned-u-s-j...,92


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
24,127,2025-11-07-02-15-00 +0000,wsj,"Republican Rep. Elise Stefanik, an ally of Pre...",https://www.wsj.com/politics/elections/elise-s...
145,67,2025-11-06-22-09-16 +0000,nypost,Tesla shareholders approve Elon Musk’s histori...,https://nypost.com/2025/11/06/business/tesla-s...
118,64,2025-11-06-22-50-15 +0000,nypost,Newark International Airport facing flight del...,https://nypost.com/2025/11/06/us-news/newark-i...
241,53,2025-11-06-17-53-57 +0000,nypost,"Zohran Mamdani ally Linda Sarsour, NYC DSA pol...",https://nypost.com/2025/11/06/us-news/zohran-m...
189,52,2025-11-06-20-38-34 +0000,nypost,Congress may not be Pelosi-free for long despi...,https://nypost.com/2025/11/06/us-news/congress...
170,49,2025-11-06-21-16-00 +0000,wsj,The Bank of Mexico extended its monetary easin...,https://www.wsj.com/articles/bank-of-mexico-cu...
155,44,2025-11-06-21-52-00 +0000,wsj,Supreme Court Lets Trump Administration Enforc...,https://www.wsj.com/politics/policy/supreme-co...
178,43,2025-11-06-20-54-00 +0000,wsj,Employers said they cut more than a million jo...,https://www.wsj.com/economy/jobs/planned-u-s-j...
56,41,2025-11-07-01-18-00 +0000,wsj,Senate Majority Leader John Thune expects Sena...,https://www.wsj.com/politics/policy/government...
88,41,2025-11-07-00-09-54 +0000,nypost,3 pilots killed in fiery UPS plane crash ID’d ...,https://nypost.com/2025/11/06/us-news/3-pilots...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
